In [1]:
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI,OpenAIEmbeddings #this to above classes we used to interact with llm model.
from dotenv import load_dotenv
from langgraph.graph import StateGraph,END,START #this class we used to build stateful workflows in graphical form mei.
from pydantic import BaseModel,Field,computed_field
from typing import Annotated,List,Dict,TypedDict,Optional

load_dotenv()

True

```
🔹 Prompt Chaining – Simple language me
Prompt chaining ka matlab hai → ek LLM ka output ko doosre LLM (ya prompt) ke input me dena.
Isse hum ek multi-step reasoning pipeline banate hain.
Har step ek node hota hai (jaise LangGraph me nodes), aur ek node ka nikalne wala result agle node ke input me jata hai.

usually kab used karte(Prompt Chaining) hai jab user query ko model mei dekar sahi structure(english) mei define karna hai 
fir usko model2 as prompt bhejkar response user ko dikhate hai
```

# step:1) defining llm model object

In [2]:
from langchain_huggingface import ChatHuggingFace,HuggingFaceEndpoint


#first we have to define kon se HuggingFaceEndpoint pe API Request jayeghi
# Hugging Face endpoint define karo
llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
)

# Chat model object banao
model1 = ChatHuggingFace(llm=llm)
model2 = ChatOpenAI(temperature=0.2)

# step:2) defining state or memory class by using pydantic after then creating stategraph class object

In [3]:
class BlogContentState(BaseModel):
    title : str = Field(...,description="Taking the blog title from user")
    outline : Optional[str] =  Field(None,description="Based on User Title generating Blog Outline")
    blog : Optional[str] =  Field(None,description="Blog content")

In [4]:
#creating stategraph class object.
graph = StateGraph(BlogContentState)
graph

In [5]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
parser = StrOutputParser()

In [6]:
#reducer function this node perform action based on task
def LLM_outline_Blog(state:BlogContentState) -> BlogContentState:
    #fetching the variable from BlogContentState object.
    title = state.title #provide by user
    
    
    #creating structure instruction prompt based on title
    prompt1 = PromptTemplate(
    template="""
    You are a helpful assistant.
    Please provide a Blog Outline to the following title:

    title: {title}""",
    input_variables=["title"])
    
    
    #creating seq chain to perform action
    chain1 = prompt1 | model1 | parser
    
    #calling or involing the chain
    outline = chain1.invoke({'title': title})
    
    #updating this outline to BlogContentState object.
    state.outline = outline
    
    return state #updated state.
    

In [7]:
#reducer function this node2 perform action based on task
def Outline_to_Blog(state:BlogContentState) -> BlogContentState:
    #fetching the variable from BlogContentState object.
    outline = state.outline #provide by user
    title = state.title
    
    
    #creating structure instruction prompt based on title
    # Prompt for blog content
    blog_prompt = PromptTemplate(
        template="""
        You are a professional content writer.
        Using the provided title and outline, write a complete blog article in Summarize Manner.
        Make sure the blog is engaging, well-organized, and easy to read.

        Title: {title}

        Outline:
        {outline}

        Blog:
        """,
        input_variables=["title", "outline"]
    )

    
    
    #creating seq chain to perform action
    chain2 = blog_prompt | model2 | parser
    
    #calling or involing the chain
    blog = chain2.invoke({'title': title,"outline":outline})
    
    #updating this outline to BlogContentState object.
    state.blog = blog
    
    return state #updated state.
    

# step:3) now adding nodes and edges to graph

In [8]:
graph.add_node("LLM_outline_Blog",LLM_outline_Blog)
graph.add_node("Outline_to_Blog",Outline_to_Blog)


#connecting the node with edges
graph.add_edge(START,"LLM_outline_Blog")
graph.add_edge("LLM_outline_Blog","Outline_to_Blog")
graph.add_edge("Outline_to_Blog",END)



# step:4) #compiling the graph and showing graph workflow

In [13]:
workflow = graph.compile()

In [14]:
#invoking the workflow.
initial_state = BlogContentState(title="Artificial Intelligence")
final_state = workflow.invoke(initial_state)
final_state

{'title': 'Artificial Intelligence',
 'outline': " Title: Artificial Intelligence\n\n1. Introduction\n   - Brief overview of Artificial Intelligence (AI)\n   - Importance and relevance of AI in today's world\n\n2. History of Artificial Intelligence\n   - Timeline of AI development\n   - Key milestones and pioneers in AI history\n\n3. Types of Artificial Intelligence\n   - Explanation of different types of AI: Narrow AI, General AI, and Superintelligent AI\n   - Real-world examples of each type\n\n4. Machine Learning and Deep Learning\n   - Explanation of Machine Learning and Deep Learning\n   - Differences between the two and their applications in AI\n\n5. AI in Everyday Life\n   - Examples of AI in consumer products, services, and entertainment\n   - Discussion on the impact of AI on daily routines\n\n6. AI in Business and Industry\n   - Explanation of AI's role in various industries such as healthcare, finance, and manufacturing\n   - Case studies of successful AI implementations in 

In [15]:
print(final_state['blog'])

Title: Artificial Intelligence

Introduction
Artificial Intelligence (AI) is a rapidly evolving technology that has become increasingly important in today's world. From improving efficiency in businesses to enhancing everyday life, AI has a wide range of applications that continue to shape our future.

History of Artificial Intelligence
The development of AI can be traced back to the 1950s, with key milestones such as the creation of the first AI program and the development of neural networks. Pioneers in AI history include Alan Turing, John McCarthy, and Marvin Minsky, who laid the foundation for the field.

Types of Artificial Intelligence
There are three main types of AI: Narrow AI, General AI, and Superintelligent AI. Narrow AI is focused on specific tasks, while General AI aims to mimic human intelligence. Superintelligent AI surpasses human intelligence and is still a theoretical concept. Real-world examples include virtual assistants like Siri, self-driving cars, and advanced ch